Import needed libraries

In [ ]:
import os
import sys
from pathlib import Path
import csv
from pydantic import BaseModel
sys.path.append("..")

from utils.llm_client import LLMClient
from utils.prompts import render
from utils.llm_client import LLMClient
from utils.login_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model


In [2]:
examples ="""
Example 1
text:
BREAKING: Mahaweli River water levels rising rapidly near Manampitiya. Flood warnings issued to nearby villages.
intent: news_update
severity: medium

Example 2
text:
SOS: Elderly couple stranded inside house in Kalutara North. Water already above waist level. Need rescue boat urgently.
intent: sos_request
severity: critical

Example 3
text:
Update: Electricity restored to most areas in Ratnapura. No casualties reported so far.
intent: news_update
severity: low

Example 4

text:
We need baby food, clean water, and blankets for the temporary camp at Matara school.
intent: resource_request
severity: high

Example 5
text:
Just heard that several roads in Badulla are blocked due to landslides. Please avoid travel unless necessary.
intent: news_update
severity: medium

Example 6
text:
SOS: Two children swept away by floodwater near bridge in Kurunegala. Last seen holding onto debris.
intent: sos_request
severity: critical

Example 7
text:
Galle town experiencing heavy rain. Streets partially flooded but shops still open.
intent: news_update
severity: low

"""

In [4]:
model = pick_model("groq", "general")
llm_client = LLMClient("groq", model)

with open("../data/Sample Messages.txt", "r", encoding="utf-8") as f:
    texts = [line.strip() for line in f if line.strip()]

for text in texts:
    prompt_text, spec = render(
        "few_shot.v1",
        role="SOS call classifier",
        examples=examples,
        query=f"Review this message: {text}",
        constraints=(
            "Follow the pattern in examples. "
            "District is specified,Not mentioned if not specified"
            "Intent must be one of Rescue, Supply, Info, Other. "
            "Priority must be Low, Medium, or High."
        ),
        format="District: {district}\nIntent: {intent}\nPriority: {priority}",
    )

    messages = [{"role": "user", "content": prompt_text}]
    response = llm_client.chat(messages, temperature=0.2)

    response_text = response["text"]

    parsed = {}
    for line in response_text.splitlines():
        if ":" in line:
            key, value = line.split(":", 1)
            parsed[key.strip()] = value.strip()

    district = parsed.get("District", "")
    intent = parsed.get("Intent", "")
    priority = parsed.get("Priority", "")

    with open("../output/classified_messages.csv", mode="a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        if f.tell() == 0:
            writer.writerow(["District", "Intent", "Priority"])

        writer.writerow([district, intent, priority])
